In [1]:
import requests
import pandas as pd
from datetime import datetime
import os

In [4]:
def convert_date(date_str):
    """
    날짜 문자열을 datetime으로 변환하는 함수
    유효하지 않은 날짜는 None으로 처리
    """
    try:
        if pd.isna(date_str) or date_str == '' or date_str == '0':
            return None
        if date_str == '99991231':
            return None
        return pd.to_datetime(str(date_str), format='%Y%m%d')
    except:
        return None

In [5]:
def get_rent_loan_products(api_key):
    """
    금융감독원 전세자금대출 상품 정보를 가져와서 DataFrame으로 변환하는 함수
    """

    base_url = "http://finlife.fss.or.kr/finlifeapi/rentHouseLoanProductsSearch.json"
    params = {
        "auth": api_key,
        "topFinGrpNo": "020000",
        "pageNo": "1"
    }

    try:
        # API 요청
        response = requests.get(base_url, params=params)
        response.raise_for_status()
        data = response.json()

        # 응답 데이터 확인
        print("API 응답 성공")

        base_list = data.get('result', {}).get('baseList', [])
        option_list = data.get('result', {}).get('optionList', [])

        if not base_list or not option_list:
            print("데이터가 비어있습니다")
            return None

        df_base = pd.DataFrame(base_list)
        df_option = pd.DataFrame(option_list)

        # 컬럼 존재 여부 확인
        required_columns = ['dcls_strt_day', 'dcls_end_day']
        for col in required_columns:
            if col not in df_base.columns:
                print(f"'{col}' 컬럼이 응답 데이터에 없습니다")
                print("사용 가능한 컬럼:", df_base.columns.tolist())

        # 기본 정보 컬럼명 한글로 변경
        base_column_mapping = {
            'fin_co_no': '금융회사코드',
            'kor_co_nm': '금융회사명',
            'fin_prdt_cd': '금융상품코드',
            'fin_prdt_nm': '금융상품명',
            'join_way': '가입방법',
            'loan_inci_expn': '대출부대비용',
            'erly_rpay_fee': '중도상환수수료',
            'dly_rate': '연체이자율',
            'loan_lmt': '대출한도',
            'dcls_strt_day': '공시시작일',
            'dcls_end_day': '공시종료일',
            'join_member': '가입대상',
            'max_limit': '최대한도',
            'min_limit': '최소한도'
        }

        # 옵션 정보 컬럼명 한글로 변경
        option_column_mapping = {
            'fin_co_no': '금융회사코드',
            'fin_prdt_cd': '금융상품코드',
            'rpay_type': '대출상환유형코드',
            'rpay_type_nm': '대출상환유형',
            'lend_rate_type': '금리구분코드',
            'lend_rate_type_nm': '금리구분',
            'lend_rate_min': '금리_최저',
            'lend_rate_max': '금리_최고',
            'lend_rate_avg': '금리_평균'
        }

        # 존재하는 컬럼만 매핑
        base_mapping = {k: v for k, v in base_column_mapping.items() if k in df_base.columns}
        option_mapping = {k: v for k, v in option_column_mapping.items() if k in df_option.columns}

        df_base = df_base.rename(columns=base_mapping)
        df_option = df_option.rename(columns=option_mapping)

        # 날짜 데이터 처리
        if '공시시작일' in df_base.columns:
            df_base['공시시작일'] = df_base['공시시작일'].apply(convert_date)
        if '공시종료일' in df_base.columns:
            df_base['공시종료일'] = df_base['공시종료일'].apply(convert_date)

        # 현재 유효한 상품 표시
        current_date = pd.Timestamp.now()
        df_base['현재유효여부'] = False
        if '공시시작일' in df_base.columns and '공시종료일' in df_base.columns:
            df_base['현재유효여부'] = (
                (df_base['공시시작일'].notna()) &
                ((df_base['공시종료일'].isna()) | (df_base['공시종료일'] >= current_date))
            )

        # 숫자형 데이터 처리
        numeric_columns = ['금리_최저', '금리_최고', '금리_평균']
        for col in numeric_columns:
            if col in df_option.columns:
                df_option[col] = pd.to_numeric(df_option[col], errors='coerce')

        # DataFrame 병합
        df_merged = pd.merge(
            df_base,
            df_option,
            on=['금융회사코드', '금융상품코드'],
            how='left'
        )

        # 컬럼 순서 정의
        column_order = [
            '금융회사명', '금융상품명', '가입대상', '대출상환유형',
            '금리구분', '금리_최저', '금리_최고', '금리_평균',
            '최소한도', '최대한도', '대출한도',
            '대출부대비용', '중도상환수수료', '연체이자율',
            '가입방법', '공시시작일', '공시종료일', '현재유효여부',
            '금융회사코드', '금융상품코드', '대출상환유형코드', '금리구분코드'
        ]

        # 존재하는 컬럼만 선택
        final_columns = [col for col in column_order if col in df_merged.columns]
        df_merged = df_merged[final_columns]

        print("데이터 처리 완료")
        return df_merged

    except requests.exceptions.RequestException as e:
        print(f"API 요청 중 오류 발생: {e}")
        return None
    except Exception as e:
        print(f"데이터 처리 중 오류 발생: {e}")
        print(f"오류 상세: {str(e)}")
        return None

In [6]:
API_KEY = ""  # 발급받은 API 키를 입력하세요

    # 데이터 수집
df = get_rent_loan_products(API_KEY)

if df is not None:
    print("\n=== 데이터 수집 결과 ===")
    print(f"총 상품 수: {len(df)}")

    # 컬럼 목록 출력
    print("\n사용 가능한 컬럼:")
    print(df.columns.tolist())

    # 데이터 미리보기
    preview_columns = [col for col in ['금융회사명', '금융상품명', '금리_최저', '금리_최고'] if col in df.columns]
    print("\n=== 데이터 미리보기 ===")
    print(df[preview_columns].head())

    # CSV 저장
    #df.to_csv('rent_loan_products.csv', index=False, encoding='utf-8-sig')
    #print("\nCSV 파일 저장 완료: rent_loan_products.csv")

API 응답 성공
데이터 처리 완료

=== 데이터 수집 결과 ===
총 상품 수: 80

사용 가능한 컬럼:
['금융회사명', '금융상품명', '대출상환유형', '금리구분', '금리_최저', '금리_최고', '금리_평균', '대출한도', '대출부대비용', '중도상환수수료', '연체이자율', '가입방법', '공시시작일', '공시종료일', '현재유효여부', '금융회사코드', '금융상품코드', '대출상환유형코드', '금리구분코드']

=== 데이터 미리보기 ===
  금융회사명          금융상품명  금리_최저  금리_최고
0  우리은행  우리전세론(주택금융보증)   4.42   5.42
1  우리은행  우리전세론(주택금융보증)   4.26   5.26
2  우리은행  우리전세론(주택금융보증)   4.42   5.42
3  우리은행  우리전세론(주택금융보증)   4.26   5.26
4  우리은행    우리전세론(서울보증)   4.61   5.61


In [ ]:
# 사용 예시:

# # 1. 데이터 수집
# df = get_rent_loan_products(API_KEY)

# # 2. 데이터 확인
# if df is not None:
#     # 기본 정보 확인
#     print(df.info())

#     # 특정 컬럼만 선택해서 보기
#     selected_columns = ['금융회사명', '금융상품명', '금리_최저', '금리_최고']
#     print(df[selected_columns].head())

#     # CSV로 저장
#     df.to_csv('전세자금대출상품.csv', index=False, encoding='utf-8-sig')